**Import Libraries**

In [ ]:
import pandas as pd
import psycopg2

**Connect to the Database**

In [ ]:
# DB connection setup using hardcoded credentials
conn = psycopg2.connect(
    dbname="neondb",
    user="neondb_owner",
    password="npg_CeS9fJg2azZD",
    host="ep-falling-glitter-a5m0j5gk-pooler.us-east-2.aws.neon.tech",
    port="5432",
    sslmode="require"
)
cur = conn.cursor()

**Run a Test Query**

In [ ]:
query = "SELECT * FROM nyc_schools.high_school_directory LIMIT 5;"
df = pd.read_sql(query, conn)
df.head()

<ipython-input-9-1731849884>:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,dbn,school_name,borough,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,...,number_programs,Location 1,Community Board,Council District,Census Tract,Zip Codes,Community Districts,Borough Boundaries,City Council Districts,Police Precincts
0,27Q260,Frederick Douglass Academy VI High School,Queens,Q465,718-471-2154,718-471-2890,9.0,12,None,None,...,1,"{'latitude': '40.601989336', 'longitude': '-73...",14,31,100802,20529,51,3,47,59
1,21K559,Life Academy High School for Film and Music,Brooklyn,K400,718-333-7750,718-333-7775,9.0,12,None,None,...,1,"{'latitude': '40.593593811', 'longitude': '-73...",13,47,306,17616,21,2,45,35
2,16K393,Frederick Douglass Academy IV Secondary School,Brooklyn,K026,718-574-2820,718-574-2821,9.0,12,None,None,...,1,"{'latitude': '40.692133704', 'longitude': '-73...",3,36,291,18181,69,2,49,52
3,08X305,Pablo Neruda Academy,Bronx,X450,718-824-1682,718-824-1663,9.0,12,None,None,...,1,"{'latitude': '40.822303765', 'longitude': '-73...",9,18,16,11611,58,5,31,26
4,03M485,Fiorello H. LaGuardia High School of Music & A...,Manhattan,M485,212-496-0700,212-724-5748,9.0,12,None,None,...,6,"{'latitude': '40.773670507', 'longitude': '-73...",7,6,151,12420,20,4,19,12


**Question 1: Number of schools in each district**

In [ ]:
query_1 = """
SELECT borough, COUNT(DISTINCT dbn) AS school_count
FROM nyc_schools.high_school_directory
GROUP BY borough
ORDER BY borough;
"""

df_schools = pd.read_sql(query_1, conn)
df_schools

<ipython-input-16-2447622278>:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_schools = pd.read_sql(query_1, conn)


,borough,school_count
0,Bronx,118
1,Brooklyn,121
2,Manhattan,106
3,Queens,80
4,Staten Island,10


**Question 2: Average percentage of ELLs by district**

In [ ]:
query_2 = """
SELECT hd.borough, AVG(sd.ell_percent) AS avg_ell_percent
FROM nyc_schools.school_demographics sd
JOIN nyc_schools.high_school_directory hd ON sd.dbn = hd.dbn
GROUP BY hd.borough
ORDER BY hd.borough;
"""

df_ell = pd.read_sql(query_2, conn)
df_ell

<ipython-input-17-3528288731>:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_ell = pd.read_sql(query_2, conn)


,borough,avg_ell_percent
0,Manhattan,7.5725


**Question 3: Top 3 schools by sped_percent**

In [ ]:
query_3 = """
SELECT hd.borough, sd.dbn, hd.school_name, sd.sped_percent
FROM nyc_schools.school_demographics sd
JOIN nyc_schools.high_school_directory hd ON sd.dbn = hd.dbn
WHERE sd.sped_percent IS NOT NULL
ORDER BY hd.borough, sd.sped_percent DESC;
"""

df_sped = pd.read_sql(query_3, conn)
top3_sped = df_sped.groupby("borough").head(3).reset_index(drop=True)
top3_sped

<ipython-input-18-2054955530>:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sped = pd.read_sql(query_3, conn)


,borough,dbn,school_name,sped_percent
0,Manhattan,01M450,East Side Community School,28.8
1,Manhattan,01M450,East Side Community School,27.7
2,Manhattan,01M450,East Side Community School,26.7
